### GAN Attempt!

In [ ]:
!pip install transformers
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 45.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 57.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 11.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 38.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 27.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 21.0 MB/s eta 0:00:0

In [ ]:
"""
Setup
"""

import torch
import torch.nn as nn
import torch.optim as optim
from transformers import (
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
)
import pandas as pd
from datasets import Dataset

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

%cd drive/MyDrive/CS\ 224N\ Project
%ls # verify that you are in the right directory

Mounted at /content/drive
/content/drive/.shortcut-targets-by-id/1x7bmzM_qtbj3CPKzenuwvwocK9KiZzWU/CS 224N Project
'224N Experiments (GAN on 998 Samples).gsheet'
'224N Project Brainstorm.gdoc'
'224N Project Helpful Tutorials.gdoc'@
'224N Project Milestone Notes.gdoc'
 Adversarial-T5_Structure_1.ipynb
 aita_clean.csv
 aita_comments.csv
 aita_test_set.csv
 aita_test_set.gsheet
 aita_train_set.csv
 aita_valid_set.csv
 banana.ipynb
 bart-base-checkpoint-204000/
 bart-baseline-attempt2.ipynb
 bart-checkpoint-5000/
 bert-baseline.ipynb
 checkpoint.txt
 config.json
 csvs/
 dataset_agg.ipynb
 drive/
 Evaluate.ipynb
'experimenting with gumbel.ipynb'
 finetune-gpt2/
'first proposal OLD'/
 gan-gen-trial
 gpt-2-attempt2.ipynb
 gpt2-attempt3.ipynb
 gpt2-baseline.ipynb
 gpt2-small-rationale-generation/
 gpt2-wt-5.ipynb
 grape/
 grape.ipynb
 logs/
 mango/
 Mango.ipynb
 Orange.ipynb
 Pear.ipynb
 pineapple/
 Pineapple.ipynb
 processed-set-gpt2/
'Reddit Scraper.ipynb'
 results/
 rouge_scores.txt
'screen

In [ ]:
"""
Define the generator (use the pre-trained BART implementation)
"""

# bart-base checkpoint pre-trained on our dataset
# (can also try generically pre-trained bart base)
model_dir = 'bart-base-checkpoint-204000'
tokenizer = AutoTokenizer.from_pretrained(model_dir)
netG = AutoModelForSeq2SeqLM.from_pretrained(model_dir)
print(netG)

BartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(50265, 768, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(50265, 768, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 768)
      (layers): ModuleList(
        (0): BartEncoderLayer(
          (self_attn): BartAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_norm): LayerNorm((768,), eps=1e-05,

In [ ]:
"""
Define convolutional discriminator
"""

nc = 1
ndf = 64

class Discriminator(nn.Module):
    def __init__(self, ngpu):
        super(Discriminator, self).__init__()
        self.ngpu = ngpu
        self.main = nn.Sequential(
            #Reshaping input
            nn.Upsample(size=(64, 64)), #bring image from 1, 1, 1, 64 --> 1, 1, 64, 64
            # input is (nc) x 64 x 64 | our input is 1 x 64
            nn.Conv2d(nc, ndf, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf) x 32 x 32
            nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*2) x 16 x 16
            nn.Conv2d(ndf * 2, ndf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 4),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*4) x 8 x 8
            nn.Conv2d(ndf * 4, ndf * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 8),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*8) x 4 x 4
            nn.Conv2d(ndf * 8, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
        )

    def forward(self, input):
        #print(len(input.shape))
        return self.main(input)

In [ ]:
ngpu = 1
netD = Discriminator(ngpu).to(device)

In [ ]:
"""
Loss functions and optimizers
"""
# Size of generator input
nz = 512
# Optim params
lr = 0.0002
beta1 = 0.5

# Initialize BCELoss function
criterion = nn.BCELoss()

# Create batch of latent vectors that we will use to visualize
#  the progression of the generator
fixed_noise = torch.randn(64, nz, 1, 1, device=device)

# Establish convention for real and fake labels during training
real_label = 1.
fake_label = 0.

# Setup Adam optimizers for both G and D
optimizerD = optim.Adam(netD.parameters(), lr=lr, betas=(beta1, 0.999))
optimizerG = optim.Adam(netG.parameters(), lr=lr, betas=(beta1, 0.999))

In [ ]:
train_df = pd.read_csv('aita_train_set.csv')[['text', 'comments']]
valid_df = pd.read_csv('aita_valid_set.csv')[['text', 'comments']]
test_df = pd.read_csv('aita_test_set.csv')[['text', 'comments']]

In [ ]:
train_data_txt = Dataset.from_pandas(train_df)
validation_data_txt = Dataset.from_pandas(valid_df)
test_data_txt = Dataset.from_pandas(test_df)
print(train_data_txt)
print(validation_data_txt)
print(test_data_txt)

Dataset({
    features: ['text', 'comments'],
    num_rows: 81614
})
Dataset({
    features: ['text', 'comments'],
    num_rows: 998
})
Dataset({
    features: ['text', 'comments'],
    num_rows: 998
})


In [ ]:
"""
Preprocess
"""

encoder_max_length = 256  # changed from 256
decoder_max_length = 64  # changed from 64

def batch_tokenize_preprocess(batch, tokenizer, max_source_length, max_target_length):
    source, target = batch["text"], batch["comments"]
    source_tokenized = tokenizer(
        source, padding="max_length", truncation=True, max_length=max_source_length, return_tensors="pt"
    )
    target_tokenized = tokenizer(
        target, padding="max_length", truncation=True, max_length=max_target_length, return_tensors="pt"
    )

    batch = {k: v for k, v in source_tokenized.items()}
    # Ignore padding in the loss
    batch["labels"] = [
        [-100 if token == tokenizer.pad_token_id else token for token in l]
        for l in target_tokenized["input_ids"]
    ]
    return batch


train_data = train_data_txt.map(
    lambda batch: batch_tokenize_preprocess(
        batch, tokenizer, encoder_max_length, decoder_max_length
    ),
    batched=True,
    remove_columns=train_data_txt.column_names,
)

validation_data = validation_data_txt.map(
    lambda batch: batch_tokenize_preprocess(
        batch, tokenizer, encoder_max_length, decoder_max_length
    ),
    batched=True,
    remove_columns=validation_data_txt.column_names,
)

Map:   0%|          | 0/81614 [00:00<?, ? examples/s]

Map:   0%|          | 0/998 [00:00<?, ? examples/s]

In [ ]:
print(train_data)

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 81614
})


In [ ]:
print(len(train_data[0]['input_ids']))

256


In [ ]:
fixed_validation_index = 17
fixed_validation_inputs = valid_df.iloc[fixed_validation_index]['text']
fixed_validation_data = tokenizer(fixed_validation_inputs, max_length=512, padding='max_length', truncation=True, return_tensors="pt")

In [ ]:
# batch_size=4
# dataloader = torch.utils.data.DataLoader(train_data, batch_size=batch_size,
#                                          shuffle=True)
# for i, data in enumerate(dataloader, 0):
#   print(torch.stack(data['attention_mask']).shape)
#   break

In [ ]:
################ Changing the working data to validation ####################

# Training Loop

# Lists to keep track of progress
img_list = []
G_losses = []
D_losses = []
iters = 0
num_epochs = 1
max_input_length = 512

print("Starting Training Loop...")
# For each epoch
for epoch in range(num_epochs):
    # todo: batch this/use a dataloader
    for i in range(len(validation_data)):
        data = validation_data[i]
        ############################
        # (1) Update D network: maximize log(D(x)) + log(1 - D(G(z)))
        ###########################
        ## Train with all-real batch
        netD.zero_grad()
        # Format batch
        #print(len(data['labels']))
        real_cpu = torch.tensor(data['labels'], dtype=torch.float32)
        #real_cpu = data['labels']
        #print(real_cpu.shape)
        # real_cpu = real_cpu.unsqueeze(0)
        # real_cpu = real_cpu.unsqueeze(0)
        # real_cpu = real_cpu.unsqueeze(0)
        real_cpu = real_cpu.view(1, 1, 1, 64) #these are the comment tokens
        #print(real_cpu.shape)
        real_cpu = real_cpu.to(device)
        b_size = real_cpu.size(0)
        label = torch.full((b_size,), real_label, dtype=torch.float, device=device)

        #discriminator will train off of true comments in the real batch pass
        # Forward pass real batch through D
        output = netD(real_cpu).view(-1) 
        # Calculate loss on all-real batch
        errD_real = criterion(output, label)
        # Calculate gradients for D in backward pass
        errD_real.backward()
        D_x = output.mean().item()

        ## Train with all-fake batch
        # Generate batch of latent vectors
        # noise = torch.randn(b_size, nz, 1, 1, device=device)
        # print(inputs['input_ids'].shape)
        # output = model.generate(**inputs, num_beams=8, do_sample=True, min_length=10, max_length=64)
        # Generate fake image batch with G

        inputs = valid_df.iloc[i]['text']
        data = tokenizer(inputs, max_length=max_input_length, padding='max_length', truncation=True, return_tensors="pt")
        fake = netG.generate(**data, num_beams=8, do_sample=True, min_length=10, max_length=64) #generate a fake comment
        label.fill_(fake_label)
        # Classify all fake batch with D
        #print(fake.shape)
        fake = fake.type(torch.float32)
        fake = fake.view(1, 1, 1, -1)
        fake = fake.detach().to(device)
        output = netD(fake).view(-1)
        # Calculate D's loss on the all-fake batch
        errD_fake = criterion(output, label)
        # Calculate the gradients for this batch, accumulated (summed) with previous gradients
        errD_fake.backward()
        D_G_z1 = output.mean().item()
        # Compute error of D as sum over the fake and the real batches
        errD = errD_real + errD_fake
        # Update D
        optimizerD.step()

        ############################
        # (2) Update G network: maximize log(D(G(z)))
        ###########################
        netG.zero_grad()
        label.fill_(real_label)  # fake labels are real for generator cost
        # Since we just updated D, perform another forward pass of all-fake batch through D
        output = netD(fake).view(-1)
        # Calculate G's loss based on this output
        errG = criterion(output, label)
        # Calculate gradients for G
        errG.backward()
        D_G_z2 = output.mean().item()
        # Update G
        optimizerG.step()

        # Output training stats
        if i % 5 == 0:
            print('[%d/%d][%d/%d]\tLoss_D: %.4f\tLoss_G: %.4f\tD(x): %.4f\tD(G(z)): %.4f / %.4f'
                  % (epoch, num_epochs, i, len(validation_data),
                     errD.item(), errG.item(), D_x, D_G_z1, D_G_z2))

        # Save Losses for plotting later
        G_losses.append(errG.item())
        D_losses.append(errD.item())

        if iters == 5: netG.save_pretrained('grape/hf-save-initial')
        # Check how the generator is doing by saving G's output on fixed_noise
        if (iters != 0 and iters % 500 == 0) or ((epoch == num_epochs-1) and (i == len(train_data)-1)):
            netG.save_pretrained('grape/hf-save-halfway')
            with torch.no_grad():
                fake = netG.generate(**fixed_validation_data, num_beams=8, do_sample=True, min_length=10, max_length=64).detach()
            # img_list.append(vutils.make_grid(fake, padding=2, normalize=True))

        iters += 1

netG.save_pretrained('grape/hf-save-final')

Starting Training Loop...
[0/1][0/998]	Loss_D: 1.4483	Loss_G: 5.8057	D(x): 0.3625	D(G(z)): 0.3517 / 0.0030
[0/1][5/998]	Loss_D: 4.1567	Loss_G: 3.9315	D(x): 0.0258	D(G(z)): 0.3936 / 0.0196
[0/1][10/998]	Loss_D: 2.4321	Loss_G: 3.2507	D(x): 0.0931	D(G(z)): 0.0568 / 0.0387
[0/1][15/998]	Loss_D: 0.2008	Loss_G: 5.8874	D(x): 0.8206	D(G(z)): 0.0030 / 0.0028
[0/1][20/998]	Loss_D: 3.7579	Loss_G: 2.2513	D(x): 0.9819	D(G(z)): 0.9762 / 0.1053
[0/1][25/998]	Loss_D: 1.9715	Loss_G: 4.5579	D(x): 0.4468	D(G(z)): 0.6883 / 0.0105
[0/1][30/998]	Loss_D: 3.1629	Loss_G: 4.3383	D(x): 0.3284	D(G(z)): 0.8712 / 0.0131
[0/1][35/998]	Loss_D: 2.7175	Loss_G: 3.0825	D(x): 0.2276	D(G(z)): 0.7099 / 0.0458
[0/1][40/998]	Loss_D: 1.9320	Loss_G: 2.4351	D(x): 0.1958	D(G(z)): 0.2602 / 0.0876
[0/1][45/998]	Loss_D: 4.2772	Loss_G: 3.1137	D(x): 0.0463	D(G(z)): 0.7001 / 0.0444
[0/1][50/998]	Loss_D: 0.4257	Loss_G: 3.5196	D(x): 0.8638	D(G(z)): 0.2437 / 0.0296
[0/1][55/998]	Loss_D: 2.0118	Loss_G: 2.4506	D(x): 0.2606	D(G(z)): 0.4868 /

In [ ]:
# # Training Loop

# # Lists to keep track of progress
# img_list = []
# G_losses = []
# D_losses = []
# iters = 0
# num_epochs = 1
# max_input_length = 512

# print("Starting Training Loop...")
# # For each epoch
# for epoch in range(num_epochs):
#     # todo: batch this/use a dataloader
#     for i, data in enumerate(dataloader, 0):
#         #data = train_data[i]
#         ############################
#         # (1) Update D network: maximize log(D(x)) + log(1 - D(G(z)))
#         ###########################
#         ## Train with all-real batch
#         netD.zero_grad()
#         # Format batch
#         #print(len(data['labels']))
#         #print(data)
#         print(data['labels'])
#         real_cpu = torch.stack(data['labels'])
#         #real_cpu = torch.unsqueeze(real_cpu, dim=0)
#         #real_cpu = torch.cat(real_cpu, dim=0)
#         #real_cpu = torch.tensor(data['labels'], dtype=torch.float32)
#         #real_cpu = data['labels']
#         #print(real_cpu.shape)
#         # real_cpu = real_cpu.unsqueeze(0)
#         # real_cpu = real_cpu.unsqueeze(0)
#         # real_cpu = real_cpu.unsqueeze(0)
#         real_cpu = real_cpu.type(torch.FloatTensor).view(-1, 1, 1, 64) #these are the comment tokens
#         #print(real_cpu.shape)
#         real_cpu = real_cpu.to(device)
#         b_size = real_cpu.size(0)
#         label = torch.full((b_size,), real_label, dtype=torch.float, device=device)

#         #discriminator will train off of true comments in the real batch pass
#         # Forward pass real batch through D
#         output = netD(real_cpu).view(-1) 
#         # Calculate loss on all-real batch
#         errD_real = criterion(output, label)
#         # Calculate gradients for D in backward pass
#         errD_real.backward()
#         D_x = output.mean().item()

#         ## Train with all-fake batch
#         # Generate batch of latent vectors
#         # noise = torch.randn(b_size, nz, 1, 1, device=device)
#         # print(inputs['input_ids'].shape)
#         # output = model.generate(**inputs, num_beams=8, do_sample=True, min_length=10, max_length=64)
#         # Generate fake image batch with G

#         #inputs = train_df.iloc[i]['text']
#         #data = tokenizer(inputs, max_length=max_input_length, padding='max_length', truncation=True, return_tensors="pt")
#         data = {k : torch.stack(v) for k, v in data.items()}
#         fake = netG(**data)

#         # fake = netG.generate(**data, num_beams=8, do_sample=True, min_length=10, max_length=64) #generate a fake comment
#         label.fill_(fake_label)
#         # Classify all fake batch with D
#         #print(fake.shape)
#         fake = fake.type(torch.float32)
#         fake = fake.view(1, 1, 1, -1)
#         fake = fake.detach().to(device)
#         output = netD(fake).view(-1)
#         # Calculate D's loss on the all-fake batch
#         errD_fake = criterion(output, label)
#         # Calculate the gradients for this batch, accumulated (summed) with previous gradients
#         errD_fake.backward()
#         D_G_z1 = output.mean().item()
#         # Compute error of D as sum over the fake and the real batches
#         errD = errD_real + errD_fake
#         # Update D
#         optimizerD.step()

#         ############################
#         # (2) Update G network: maximize log(D(G(z)))
#         ###########################
#         netG.zero_grad()
#         label.fill_(real_label)  # fake labels are real for generator cost
#         # Since we just updated D, perform another forward pass of all-fake batch through D
#         output = netD(fake).view(-1)
#         # Calculate G's loss based on this output
#         errG = criterion(output, label)
#         # Calculate gradients for G
#         errG.backward()
#         D_G_z2 = output.mean().item()
#         # Update G
#         optimizerG.step()

#         # Output training stats
#         if i % 5 == 0:
#             print('[%d/%d][%d/%d]\tLoss_D: %.4f\tLoss_G: %.4f\tD(x): %.4f\tD(G(z)): %.4f / %.4f'
#                   % (epoch, num_epochs, i, len(train_data),
#                      errD.item(), errG.item(), D_x, D_G_z1, D_G_z2))

#         # Save Losses for plotting later
#         G_losses.append(errG.item())
#         D_losses.append(errD.item())

#         # Check how the generator is doing by saving G's output on fixed_noise
#         if (iters % 500 == 0) or ((epoch == num_epochs-1) and (i == len(train_data)-1)):
#             with torch.no_grad():
#                 fake = netG.generate(**fixed_validation_data, num_beams=8, do_sample=True, min_length=10, max_length=64).detach()
#             # img_list.append(vutils.make_grid(fake, padding=2, normalize=True))

#         iters += 1

Starting Training Loop...
[tensor([0, 0, 0, 0]), tensor([ 4444,  7199, 34956,   487]), tensor([ 725,    5,    4, 3847]), tensor([    4, 43870,   370,   111]), tensor([1437,    6,  206,  370]), tensor([1437,   53,    5,  399]), tensor([2486,    5, 2178,   75]), tensor([2761,  129,   16,  216]), tensor([4428,  169,   22,   14]), tensor([ 101,   47, 3463,   79]), tensor([   10,   214, 37013,   938]), tensor([269, 164, 113,  75]), tensor([4613,    7,    8,  602]), tensor([621,  28, 110,  39]), tensor([   6,  441, 2472,   94]), tensor([ 53,   7,  16, 766]), tensor([ 38, 542,   7,   8]), tensor([  218,  4506, 10914,    24]), tensor([75, 14, 24, 21]), tensor([  206, 10342,    55,    10]), tensor([   24,    16, 10727,   205]), tensor([  817,     7, 38564,  4085]), tensor([  47,  464,  116, 1114]), tensor([  10,  110, 1437,    4]), tensor([ 1099,   346, 50118,   318]), tensor([  621,     8, 50118,    47]), tensor([ 114,   45,  243, 3127]), tensor([ 47, 492,  18, 399]), tensor([218,  24,  10,  7

ValueError: ignored

In [ ]:
# ################ WORKING DO NOT TOUCH ####################

# # Training Loop

# # Lists to keep track of progress
# img_list = []
# G_losses = []
# D_losses = []
# iters = 0
# num_epochs = 1
# max_input_length = 512

# print("Starting Training Loop...")
# # For each epoch
# for epoch in range(num_epochs):
#     # todo: batch this/use a dataloader
#     for i in range(len(train_data)):
#         data = train_data[i]
#         ############################
#         # (1) Update D network: maximize log(D(x)) + log(1 - D(G(z)))
#         ###########################
#         ## Train with all-real batch
#         netD.zero_grad()
#         # Format batch
#         #print(len(data['labels']))
#         real_cpu = torch.tensor(data['labels'], dtype=torch.float32)
#         #real_cpu = data['labels']
#         #print(real_cpu.shape)
#         # real_cpu = real_cpu.unsqueeze(0)
#         # real_cpu = real_cpu.unsqueeze(0)
#         # real_cpu = real_cpu.unsqueeze(0)
#         real_cpu = real_cpu.view(1, 1, 1, 64) #these are the comment tokens
#         #print(real_cpu.shape)
#         real_cpu = real_cpu.to(device)
#         b_size = real_cpu.size(0)
#         label = torch.full((b_size,), real_label, dtype=torch.float, device=device)

#         #discriminator will train off of true comments in the real batch pass
#         # Forward pass real batch through D
#         output = netD(real_cpu).view(-1) 
#         # Calculate loss on all-real batch
#         errD_real = criterion(output, label)
#         # Calculate gradients for D in backward pass
#         errD_real.backward()
#         D_x = output.mean().item()

#         ## Train with all-fake batch
#         # Generate batch of latent vectors
#         # noise = torch.randn(b_size, nz, 1, 1, device=device)
#         # print(inputs['input_ids'].shape)
#         # output = model.generate(**inputs, num_beams=8, do_sample=True, min_length=10, max_length=64)
#         # Generate fake image batch with G

#         inputs = train_df.iloc[i]['text']
#         data = tokenizer(inputs, max_length=max_input_length, padding='max_length', truncation=True, return_tensors="pt")
#         fake = netG.generate(**data, num_beams=8, do_sample=True, min_length=10, max_length=64) #generate a fake comment
#         label.fill_(fake_label)
#         # Classify all fake batch with D
#         #print(fake.shape)
#         fake = fake.type(torch.float32)
#         fake = fake.view(1, 1, 1, -1)
#         fake = fake.detach().to(device)
#         output = netD(fake).view(-1)
#         # Calculate D's loss on the all-fake batch
#         errD_fake = criterion(output, label)
#         # Calculate the gradients for this batch, accumulated (summed) with previous gradients
#         errD_fake.backward()
#         D_G_z1 = output.mean().item()
#         # Compute error of D as sum over the fake and the real batches
#         errD = errD_real + errD_fake
#         # Update D
#         optimizerD.step()

#         ############################
#         # (2) Update G network: maximize log(D(G(z)))
#         ###########################
#         netG.zero_grad()
#         label.fill_(real_label)  # fake labels are real for generator cost
#         # Since we just updated D, perform another forward pass of all-fake batch through D
#         output = netD(fake).view(-1)
#         # Calculate G's loss based on this output
#         errG = criterion(output, label)
#         # Calculate gradients for G
#         errG.backward()
#         D_G_z2 = output.mean().item()
#         # Update G
#         optimizerG.step()

#         # Output training stats
#         if i % 5 == 0:
#             print('[%d/%d][%d/%d]\tLoss_D: %.4f\tLoss_G: %.4f\tD(x): %.4f\tD(G(z)): %.4f / %.4f'
#                   % (epoch, num_epochs, i, len(train_data),
#                      errD.item(), errG.item(), D_x, D_G_z1, D_G_z2))

#         # Save Losses for plotting later
#         G_losses.append(errG.item())
#         D_losses.append(errD.item())

#         # Check how the generator is doing by saving G's output on fixed_noise
#         if (iters % 500 == 0) or ((epoch == num_epochs-1) and (i == len(train_data)-1)):
#             with torch.no_grad():
#                 fake = netG.generate(**fixed_validation_data, num_beams=8, do_sample=True, min_length=10, max_length=64).detach()
#             # img_list.append(vutils.make_grid(fake, padding=2, normalize=True))

#         iters += 1

In [ ]:
# import torch
# import torch.nn as nn

# input = torch.randn(1, 64).view(1, 1, 1, 64)
# print(input.shape)
# m = nn.Upsample(size=(64, 64))
# output = m(input)
# output = output.reshape((1, 64, 64))
# print(output.shape)

# up = nn.Upsample(size=(24, 24))

# x = torch.randn(1, 3, 10, 10)
# print(up(x).shape)

torch.Size([1, 1, 1, 64])
torch.Size([1, 64, 64])
torch.Size([1, 3, 24, 24])


In [ ]:
# up = nn.Upsample(size=(24, 24))

# x = torch.randn(1, 3, 10, 10)
# print(up(x).shape)

torch.Size([1, 3, 24, 24])


In [ ]:
# input = torch.randn(1, 64).view(1, 1, 1, 64)
# m = nn.Upsample(size=(64, 64))
# intermediate = m(input)
# x = nn.Flatten(0, 1)
# output = x(intermediate)
# print(input.shape)
# print(intermediate.shape)
# print(output.shape)

torch.Size([1, 1, 1, 64])
torch.Size([1, 1, 64, 64])
torch.Size([1, 64, 64])
